In [6]:
import pandas as pd
import re
from datetime import datetime, timedelta


In [7]:
df = pd.read_csv("customer_support_tickets.csv")
df.head()


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [8]:
print("Rows & Columns:", df.shape)
print("\nMissing Values:\n", df.isnull().sum())
df.sample(5)


Rows & Columns: (8469, 17)

Missing Values:
 Ticket ID                          0
Customer Name                      0
Customer Email                     0
Customer Age                       0
Customer Gender                    0
Product Purchased                  0
Date of Purchase                   0
Ticket Type                        0
Ticket Subject                     0
Ticket Description                 0
Ticket Status                      0
Resolution                      5700
Ticket Priority                    0
Ticket Channel                     0
First Response Time             2819
Time to Resolution              5700
Customer Satisfaction Rating    5700
dtype: int64


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
2660,2661,Patrick Preston,dillonsteven@example.com,28,Male,Xbox,2021-04-21,Technical issue,Installation support,I've noticed a software bug in the {product_pu...,Closed,Common market model wear.,High,Phone,2023-06-01 18:47:02,2023-05-31 22:51:02,4.0
7093,7094,Terry Morales,michaelblevins@example.net,47,Male,Nikon D,2020-03-06,Cancellation request,Refund request,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Low,Phone,2023-06-01 19:19:25,NaN,NaN
3990,3991,Julie Smith,paul73@example.net,47,Female,Samsung Galaxy,2021-07-10,Refund request,Installation support,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,High,Social media,2023-06-01 22:22:23,NaN,NaN
7621,7622,Harold Booker,janetjames@example.org,29,Other,LG Washing Machine,2021-12-10,Product inquiry,Payment issue,I'm encountering a software bug in the {produc...,Pending Customer Response,NaN,High,Email,2023-06-01 01:36:42,NaN,NaN
5093,5094,Brittany Bradley,hannah66@example.com,24,Female,Roomba Robot Vacuum,2020-09-10,Refund request,Payment issue,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Medium,Social media,2023-06-01 23:27:38,NaN,NaN


In [9]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df["clean_description"] = df["Ticket Description"].apply(clean_text)
df[["Ticket Description", "clean_description"]].head()


,Ticket Description,clean_description
0,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...
1,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...
2,I'm facing a problem with my {product_purchase...,im facing a problem with my productpurchased t...
3,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...
4,I'm having an issue with the {product_purchase...,im having an issue with the productpurchased p...


In [10]:
urgency_keywords = {
    "urgent": 3,
    "immediately": 3,
    "asap": 2
}

impact_keywords = {
    "payment": 3,
    "refund": 2,
    "billing": 2,
    "login": 2,
    "not working": 2,
    "error": 1
}

emotion_keywords = {
    "angry": 2,
    "frustrated": 2,
    "disappointed": 1,
    "unhappy": 1
}


In [11]:
def calculate_priority_score(message):
    score = 0

    for word, weight in urgency_keywords.items():
        if word in message:
            score += weight

    for word, weight in impact_keywords.items():
        if word in message:
            score += weight

    for word, weight in emotion_keywords.items():
        if word in message:
            score += weight

    return score


In [12]:
def assign_priority(score):
    if score >= 6:
        return "P0"
    elif score >= 4:
        return "P1"
    elif score >= 2:
        return "P2"
    else:
        return "P3"


In [13]:
df["priority_score"] = df["clean_description"].apply(calculate_priority_score)
df["calculated_priority"] = df["priority_score"].apply(assign_priority)

df[["clean_description", "priority_score", "calculated_priority"]].head()


,clean_description,priority_score,calculated_priority
0,im having an issue with the productpurchased p...,2,P2
1,im having an issue with the productpurchased p...,0,P3
2,im facing a problem with my productpurchased t...,0,P3
3,im having an issue with the productpurchased p...,0,P3
4,im having an issue with the productpurchased p...,0,P3


In [14]:
def get_priority_reason(message):
    reasons = []

    for word in urgency_keywords:
        if word in message:
            reasons.append(f"urgency: {word}")

    for word in impact_keywords:
        if word in message:
            reasons.append(f"impact: {word}")

    for word in emotion_keywords:
        if word in message:
            reasons.append(f"emotion: {word}")

    return ", ".join(reasons) if reasons else "general query"

df["priority_reason"] = df["clean_description"].apply(get_priority_reason)
df[["calculated_priority", "priority_reason"]].head()


,calculated_priority,priority_reason
0,P2,impact: billing
1,P3,general query
2,P3,general query
3,P3,general query
4,P3,general query


In [15]:
sla_hours = {
    "P0": 2,
    "P1": 6,
    "P2": 12,
    "P3": 24
}

df["sla_hours"] = df["calculated_priority"].map(sla_hours)


In [16]:
df["created_time"] = datetime.now()
df["due_time"] = df["created_time"] + df["sla_hours"].apply(lambda x: timedelta(hours=x))

df[["calculated_priority", "sla_hours", "due_time"]].head()


,calculated_priority,sla_hours,due_time
0,P2,12,2025-12-27 05:49:06.560700
1,P3,24,2025-12-27 17:49:06.560700
2,P3,24,2025-12-27 17:49:06.560700
3,P3,24,2025-12-27 17:49:06.560700
4,P3,24,2025-12-27 17:49:06.560700


In [17]:
print("Total Tickets:", len(df))
print("\nPriority Distribution:")
print(df["calculated_priority"].value_counts())

print("\nTop Reasons:")
print(df["priority_reason"].value_counts().head())


Total Tickets: 8469

Priority Distribution:
calculated_priority
P3    7577
P2     828
P1      61
P0       3
Name: count, dtype: int64

Top Reasons:
priority_reason
general query                   6829
impact: error                    746
impact: login, impact: error     191
impact: not working              177
urgency: urgent                  152
Name: count, dtype: int64


In [18]:
df.to_csv("final_ticket_triage_output.csv", index=False)
